<a href="https://colab.research.google.com/github/SergioNietoP/machine-learning-datasheets/blob/main/Learning%20Datasheets/Turf%20Prediction/BasicScript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DataSheet Description**

---


En este proyecto emplearemos un dataset que contiene información de los resultados de las carreras de el panorama nacional de carreras de caballos.

- **Puesto** (Clase): Posición en la que terminan la carrera **int64**
- **NombreCaballo**: Nombre de caballo  **object**
- **Peso** : Peso de caballo  **object**
- **Edad**: Edad de caballo **int64**
- **DistanciaMeta**: Distancia respecto a ganador en llegada a meta **object** (DROPEAR)
- **Mantilla**: Dorsal   **int64**
- **Dividendo** : Pago por euro apostado por ganador de carrera **object** (DROPEAR)
- **Propietario** : Cuadra Propietaria **object**
- **Preparador**: Nombre Preparador **object**
- **Jinete**: Jinete o Amazona que monta al caballo en esta carrera  **object**
- **Problemas**: Problemas que posee en esta carrera el caballo  **object**
- **UltimasActuaciones** : Posiciones en las que acaba el caballo de mas antigua a mas reciente (de izquierda a derecha)  **object** (¿COMO LA CALCULO?)
- **Fecha**: Fecha Carrera   **object**
- **Hora**: Hora carrera  **object**
- **Terreno**: Estado del terreno en carrera **object**
- **Distancia**: Longitud carrera  **float64**
- **Tipo**: Tipo de terreno en carrera  **object**
- **Categoría**: Categoria Caballos Participantes  **object**
- **SentidoHipodromo**:  (0 -> Sentido Normal, 1 -> Inverso) **int64**
- **Meteorología**: Previsión estado meteorología del día entero **object** (Tiempo3)
- **Lluvia**: Previsión cantidad de lluvia del dia entero  **object**
- **Viento**: Previsión velocidad de viento del dia entero  **int64**
- **TemperaturaMax**: Previsión de temp. máxima del día entero  **int64**
- **TemperaturaMin**: Previsión de temp. mínima del dia entero  **int64**
- **Hipodromo**: Nombre del hipódromo de la carrera   **object**

In [532]:
import pandas as pd
import numpy as np
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import sklearn.compose
from xgboost import XGBClassifier
from sklearn.metrics import f1_score

url = 'https://raw.githubusercontent.com/SergioNietoP/machine-learning-datasheets/main/Learning%20Datasheets/Turf%20Prediction/Data%20Turf%20Races%20Train%20-%204.csv'

url1 = 'https://raw.githubusercontent.com/SergioNietoP/machine-learning-datasheets/main/Learning%20Datasheets/Turf%20Prediction/Data%20Turf%20Races%20Test%20-%202.csv'
test = pd.read_csv(url1, sep=',')


train = pd.read_csv(url, sep=',')

train = train.drop('DistanciaMeta', axis=1)
train = train.drop('Dividendo', axis=1)

ogData = train
train.head()
train.count()



Puesto                501
NombreCaballo         501
Peso                  501
Edad                  501
Mantilla              501
Propietario           501
Preparador            501
Jinete                501
Problemas             269
UltimasActuaciones    501
Fecha                 501
Hora                  501
Terreno               501
Distancia             501
Tipo                  500
Categoría             501
SentidoHipodromo      501
Meteorología          501
LLuvia                501
Viento                501
Temperatura           501
Hipodromo             501
dtype: int64

In [533]:
# def getDebutantes(arrayLast):
#   aux = []
#   index = 0
#   for element in arrayLast:
#     element = element.replace(',', '')
#     element = element.replace('[', '')
#     element = element[:len(element)-1]
#     if 'Debutante' in element:
#       aux.append(index)
#     index += 1
#   return aux

# #Desarrollar crear columnas en 

# def getNombres(array):
#   aux = []
#   for element in array:
#     nombre = train.at[element, 'NombreCaballo']
#     nombre = nombre.partition('(')[0]
#     aux.append(nombre)
#   return aux

# def createColumnas(array):
#   for element in array:
#     train[element] = 0
    
# t_array = test["UltimasActuaciones"]
# variable = getDebutantes(t_array)
# columnas = getNombres(variable)
# createColumnas(columnas)

# Preprocesamiento
---

In [534]:
pd.set_option('display.max_columns', None)
train.head(10)


,Puesto,NombreCaballo,Peso,Edad,Mantilla,Propietario,Preparador,Jinete,Problemas,UltimasActuaciones,Fecha,Hora,Terreno,Distancia,Tipo,Categoría,SentidoHipodromo,Meteorología,LLuvia,Viento,Temperatura,Hipodromo
0,1,SAFAGA,57,2,6,ASOC.LA TOLEDANA...,CH.DELCHER,G.GUEDJ-GAY,NaN,"[02,04,01]",13 de noviembre de 2022,11:45,H - Blando,1500,Liso,C,0,Parcialmente nublado,0.0,6,15.0,La Zarzuela
1,2,ROCK MOON,57,2,5,MEDREAL,G.ARIZKORRETA,V. JANACEK,NaN,"[04,06]",13 de noviembre de 2022,11:45,H - Blando,1500,Liso,C,0,Parcialmente nublado,0.0,6,15.0,La Zarzuela
2,3,ISOLDA,57,2,3,EL HERROJO,J.M.OSORIO,J.GELABERT,NaN,[01],13 de noviembre de 2022,11:45,H - Blando,1500,Liso,C,0,Parcialmente nublado,0.0,6,15.0,La Zarzuela
3,4,LADY CLUNIA,57,2,4,BRAZACORTA,B.RAMA,B. FAYOS,-8,"[08,15]",13 de noviembre de 2022,11:45,H - Blando,1500,Liso,C,0,Parcialmente nublado,0.0,6,15.0,La Zarzuela
4,5,DE FIESTA (IRE) (a),57,2,2,REZA PAZOOKI,O.ANAYA,R.N.VALLE,-8,"[09,11]",13 de noviembre de 2022,11:45,H - Blando,1500,Liso,C,0,Parcialmente nublado,0.0,6,15.0,La Zarzuela
5,6,ALERTA ROJA,57,2,1,LAC INTERNACIONA...,A.SOTO,C. CADEL,-3,"[07,05]",13 de noviembre de 2022,11:45,H - Blando,1500,Liso,C,0,Parcialmente nublado,0.0,6,15.0,La Zarzuela
6,1,BAILEN,57,2,3,LEONESA,J.LOPEZ,J.GELABERT,NaN,"[01,08]",13 de noviembre de 2022,12:20,H - Blando,1500,Liso,C,0,Parcialmente nublado,0.0,6,15.0,La Zarzuela
7,2,ARAKA LA KANA,57,2,2,ODISEA,J.LOPEZ,C.PEREZ,-3,[02],13 de noviembre de 2022,12:20,H - Blando,1500,Liso,C,0,Parcialmente nublado,0.0,6,15.0,La Zarzuela
8,3,WITIZA,57,2,9,LAC INTERNACIONA...,J.LOPEZ,B. FAYOS,-3,"[05,04,04]",13 de noviembre de 2022,12:20,H - Blando,1500,Liso,C,0,Parcialmente nublado,0.0,6,15.0,La Zarzuela
9,4,PERILLAN,57,2,7,E. PEREZ GUZMAN,J.A.RODRIGUEZ,R.N.VALLE,-8,"[07,14,05,01,02]",13 de noviembre de 2022,12:20,H - Blando,1500,Liso,C,0,Parcialmente nublado,0.0,6,15.0,La Zarzuela


## **Tratamiento (Puesto)(Train)**

In [535]:
train['Puesto'] = train['Puesto'].replace(95, 10) # Penalización Parado

In [536]:
train['Puesto'] = train['Puesto'].replace(94, 10) # Penalización Parado

In [537]:
train['Puesto'] = train['Puesto'].replace(97, 13) # Penalización Retirado

## **Tratamiento (NombreCaballo)(Train)**

In [538]:
def eraseBlank(list):
  aux = []
  for element in list:
    element = element.strip()
    aux.append(element)
  return aux

test['NombreCaballo'] = test['NombreCaballo'].str.partition('(')[0]
listaNombreCaballo = test['NombreCaballo'].unique().tolist()

listTestHorse= eraseBlank(listaNombreCaballo)

print(listTestHorse)

['PHILIPPO', 'PONCE DE LEON', 'EL PATER', 'BRAGNOSERA', 'BERTIZ', 'SANDRO', 'GRAZALEMA', 'NICOLASA', 'LA PERLA NEGRA', 'WHITY', 'AFRODITA', 'VADALEW', 'HEBE', 'WENDY', 'MADRE MIA', 'LADY RAZALMA', 'FAITH ROSE', "WARRIOR'S REVENGE", 'SHELBY', 'WHITE WINE', 'VITA BARELIERE', 'XILADO', 'WINTON', 'AMERICANO', 'SUPERIOR BEAUTY', 'STARSHADOW', 'FUOCO GRECO', 'SANCTI PETRI', 'ROBLON', 'WALKING TO GLORY', 'FORTUNATO', 'AUSTRALIA CAPE', 'MAITRE YODA', 'ASTURIAS', 'SEVERUS', 'ROBAYERA', 'VIKING CITY', 'KANE ORE', 'EMBAT', 'MONTERREDONDO', 'HADES', 'USI DE U', 'MEDICEAN BLUE', 'EL CANEY', 'FINELY TUNED', 'THE GAME', 'ORBAYO', 'ASTRAL', 'TRES DE TREBOL', 'SOGALINDA', 'CHUSQUEZ', 'SOFUNNY', 'LA MAL AMADA', 'ATLANTICO', 'CAROLINA WEST', 'JERY SMAIH', 'RUMBERA', 'SANS ATTENDRE', 'HIGHLAND MARKET', 'PIU BIRCH', 'ARETHA', 'IZAMAL', 'UPSDAWN', 'UPSILON']


In [539]:
listTestHorse.sort()
print(listTestHorse)

['AFRODITA', 'AMERICANO', 'ARETHA', 'ASTRAL', 'ASTURIAS', 'ATLANTICO', 'AUSTRALIA CAPE', 'BERTIZ', 'BRAGNOSERA', 'CAROLINA WEST', 'CHUSQUEZ', 'EL CANEY', 'EL PATER', 'EMBAT', 'FAITH ROSE', 'FINELY TUNED', 'FORTUNATO', 'FUOCO GRECO', 'GRAZALEMA', 'HADES', 'HEBE', 'HIGHLAND MARKET', 'IZAMAL', 'JERY SMAIH', 'KANE ORE', 'LA MAL AMADA', 'LA PERLA NEGRA', 'LADY RAZALMA', 'MADRE MIA', 'MAITRE YODA', 'MEDICEAN BLUE', 'MONTERREDONDO', 'NICOLASA', 'ORBAYO', 'PHILIPPO', 'PIU BIRCH', 'PONCE DE LEON', 'ROBAYERA', 'ROBLON', 'RUMBERA', 'SANCTI PETRI', 'SANDRO', 'SANS ATTENDRE', 'SEVERUS', 'SHELBY', 'SOFUNNY', 'SOGALINDA', 'STARSHADOW', 'SUPERIOR BEAUTY', 'THE GAME', 'TRES DE TREBOL', 'UPSDAWN', 'UPSILON', 'USI DE U', 'VADALEW', 'VIKING CITY', 'VITA BARELIERE', 'WALKING TO GLORY', "WARRIOR'S REVENGE", 'WENDY', 'WHITE WINE', 'WHITY', 'WINTON', 'XILADO']


In [540]:
test['Jinete'] = test['Jinete'].str.partition('(')[0]
lista = test['Jinete'].unique().tolist()

listTestJockey = eraseBlank(lista)

test['Preparador'] = test['Preparador'].str.partition('(')[0]
lista = test['Preparador'].unique().tolist()

listTestTrainer = eraseBlank(lista)

test['Propietario'] = test['Propietario'].str.partition('(')[0]
lista = test['Propietario'].unique().tolist()

listTestOwner = eraseBlank(lista)



In [541]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Puesto              501 non-null    int64  
 1   NombreCaballo       501 non-null    object 
 2   Peso                501 non-null    object 
 3   Edad                501 non-null    int64  
 4   Mantilla            501 non-null    int64  
 5   Propietario         501 non-null    object 
 6   Preparador          501 non-null    object 
 7   Jinete              501 non-null    object 
 8   Problemas           269 non-null    object 
 9   UltimasActuaciones  501 non-null    object 
 10  Fecha               501 non-null    object 
 11  Hora                501 non-null    object 
 12  Terreno             501 non-null    object 
 13  Distancia           501 non-null    int64  
 14  Tipo                500 non-null    object 
 15  Categoría           501 non-null    object 
 16  SentidoH

In [542]:
def changeNotFoundInListHorse(listTrain, listTest):
  index = 0
  listaApariciones = []
  for element in listTrain:
    if(element not in listTest):
      train.at[index, 'NombreCaballo'] = 'OtherHorse'
    else:
      listaApariciones.append(element)
    index += 1
  return listaApariciones


train['NombreCaballo'] = train['NombreCaballo'].str.partition('(')[0]
listTrainHorse = train['NombreCaballo'].tolist()
# listTrainHorse= eraseBlank(listTrainHorse)
listAparicionesHorse= changeNotFoundInListHorse(listTrainHorse, listTestHorse)
train['NombreCaballo'].head(35)

0     OtherHorse
1     OtherHorse
2     OtherHorse
3     OtherHorse
4     OtherHorse
5     OtherHorse
6     OtherHorse
7     OtherHorse
8     OtherHorse
9     OtherHorse
10    OtherHorse
11    OtherHorse
12    OtherHorse
13    OtherHorse
14    OtherHorse
15    OtherHorse
16    OtherHorse
17    OtherHorse
18    OtherHorse
19    OtherHorse
20    OtherHorse
21    OtherHorse
22    OtherHorse
23    OtherHorse
24    OtherHorse
25    OtherHorse
26    OtherHorse
27    OtherHorse
28    OtherHorse
29    OtherHorse
30    OtherHorse
31    OtherHorse
32    OtherHorse
33    OtherHorse
34    OtherHorse
Name: NombreCaballo, dtype: object

## **Tratamiento (Peso)**

In [543]:
train['Peso'] = train['Peso'].str.partition('-')[0]
train['Peso'] = train['Peso'].str[:2]



In [544]:
train['Peso'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

57    100
56     57
58     53
59     50
54     48
55     47
60     40
61     30
63     19
52     15
53     14
62     14
65      5
66      3
69      2
70      2
64      1
67      1
Name: Peso, dtype: int64

## **Tratamiento (Edad)**

In [545]:
train['Edad'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

2    131
3    123
4    108
5     69
6     39
7     20
8     10
9      1
Name: Edad, dtype: int64

## **Tratamiento (Mantilla)**

In [546]:
train['Mantilla'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

5     49
3     49
4     49
2     49
1     49
6     46
7     43
8     37
9     32
10    25
11    22
12    16
13    10
15     7
14     7
16     5
17     4
18     2
Name: Mantilla, dtype: int64

## **Tratamiento (Propietario)(Train)**

In [547]:
def changeNotFoundInListOwner(listTrain, listTest):
  index = 0
  listaApariciones = []
  for element in listTrain:
    if(element not in listTest):
      train.at[index, 'Propietario'] = 'OtherOwner'
    else:
      listaApariciones.append(element)
    index += 1
  return listaApariciones
    
    

#Lógica de lista de Test en "Tratamiento(NombreCaballo)"

listTrainOwner = train['Propietario'].tolist()
listTrainOwner= eraseBlank(listTrainOwner)
listAparicionesOwner = changeNotFoundInListOwner(listTrainOwner, listTestOwner)
train['Propietario'].head(15)

0       OtherOwner
1       OtherOwner
2       OtherOwner
3       OtherOwner
4       OtherOwner
5       OtherOwner
6       OtherOwner
7       OtherOwner
8       OtherOwner
9       OtherOwner
10      OtherOwner
11             4 C
12      OtherOwner
13      OtherOwner
14    AMAZING TURF
Name: Propietario, dtype: object

In [548]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Puesto              501 non-null    int64  
 1   NombreCaballo       501 non-null    object 
 2   Peso                501 non-null    object 
 3   Edad                501 non-null    int64  
 4   Mantilla            501 non-null    int64  
 5   Propietario         501 non-null    object 
 6   Preparador          501 non-null    object 
 7   Jinete              501 non-null    object 
 8   Problemas           269 non-null    object 
 9   UltimasActuaciones  501 non-null    object 
 10  Fecha               501 non-null    object 
 11  Hora                501 non-null    object 
 12  Terreno             501 non-null    object 
 13  Distancia           501 non-null    int64  
 14  Tipo                500 non-null    object 
 15  Categoría           501 non-null    object 
 16  SentidoH

## **Tratamiento (Preparador)(Train)**

In [549]:
def changeNotFoundInListTrainer(listTrain, listTest):
  index = 0
  listaApariciones = []
  for element in listTrain:
    if(element not in listTest):
      train.at[index, 'Preparador'] = 'OtherTrainer'
    else:
      listaApariciones.append(element)
    index += 1
  return listaApariciones

#Lógica de lista de Test en "Tratamiento(NombreCaballo)"


listTrainTrainer = train['Preparador'].tolist()
listTrainTrainer= eraseBlank(listTrainTrainer)
listAparicionesTrainer = changeNotFoundInListTrainer(listTrainTrainer, listTestTrainer)
train['Preparador'].head(15)

0      OtherTrainer
1     G.ARIZKORRETA
2        J.M.OSORIO
3            B.RAMA
4      OtherTrainer
5            A.SOTO
6           J.LOPEZ
7           J.LOPEZ
8           J.LOPEZ
9     J.A.RODRIGUEZ
10     OtherTrainer
11    J.A.RODRIGUEZ
12    G.ARIZKORRETA
13       M&M RACING
14    J.A.RODRIGUEZ
Name: Preparador, dtype: object

## **Tratamiento (Jinete)(Train)**

In [550]:
def changeNotFoundInListJockey(listTrain, listTest):
  index = 0
  listaApariciones = []
  for element in listTrain:
    if(element not in listTest):
      train.at[index, 'Jinete'] = 'OtherJockey'
    else:
      listaApariciones.append(element)
    index += 1
  return listaApariciones

#Lógica de lista de Test en "Tratamiento(NombreCaballo)"


listTrainJockey = train['Jinete'].tolist()
listTrainJockey= eraseBlank(listTrainJockey)
listAparicionesJockey = changeNotFoundInListJockey(listTrainJockey, listTestJockey)
train['Jinete'].head(15)

0       OtherJockey
1        V. JANACEK
2        J.GELABERT
3          B. FAYOS
4         R.N.VALLE
5          C. CADEL
6        J.GELABERT
7           C.PEREZ
8          B. FAYOS
9         R.N.VALLE
10       V. JANACEK
11     N. DE JULIAN
12    J.L. MARTINEZ
13      V.ALONSO V.
14     J.L. BORREGO
Name: Jinete, dtype: object

## **Tratamiento (Problemas)**


In [551]:
train['Problemas'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

-8           80
-3           34
8            27
-5           22
(3)(8)       18
(5)(8)       17
(4)(8)       14
3            13
-4           12
-2            6
5             5
(2)(4)(8)     4
(2)(5)(8)     4
(2)(5)        2
(2)(3)(8)     2
(2)(8)        2
(2)(4)        2
4             2
(2)(3)        1
(4)(6)        1
(3)(8)(2)     1
Name: Problemas, dtype: int64

In [552]:
train['Problemas'] = train['Problemas'].fillna('0')
train['Problema_Nulo'] = np.where(train['Problemas'].str.contains('0'),1,0)
train['Problema_1'] = np.where(train['Problemas'].str.contains('1'),1,0)
train['Problema_2'] = np.where(train['Problemas'].str.contains('2'),1,0)
train['Problema_3'] = np.where(train['Problemas'].str.contains('3'),1,0)
train['Problema_4'] = np.where(train['Problemas'].str.contains('4'),1,0)
train['Problema_5'] = np.where(train['Problemas'].str.contains('5'),1,0)
train['Problema_6'] = np.where(train['Problemas'].str.contains('6'),1,0)
train['Problema_7'] = np.where(train['Problemas'].str.contains('7'),1,0)
train['Problema_8'] = np.where(train['Problemas'].str.contains('8'),1,0)
train.head()



,Puesto,NombreCaballo,Peso,Edad,Mantilla,Propietario,Preparador,Jinete,Problemas,UltimasActuaciones,Fecha,Hora,Terreno,Distancia,Tipo,Categoría,SentidoHipodromo,Meteorología,LLuvia,Viento,Temperatura,Hipodromo,Problema_Nulo,Problema_1,Problema_2,Problema_3,Problema_4,Problema_5,Problema_6,Problema_7,Problema_8
0,1,OtherHorse,57,2,6,OtherOwner,OtherTrainer,OtherJockey,0,"[02,04,01]",13 de noviembre de 2022,11:45,H - Blando,1500,Liso,C,0,Parcialmente nublado,0.0,6,15.0,La Zarzuela,1,0,0,0,0,0,0,0,0
1,2,OtherHorse,57,2,5,OtherOwner,G.ARIZKORRETA,V. JANACEK,0,"[04,06]",13 de noviembre de 2022,11:45,H - Blando,1500,Liso,C,0,Parcialmente nublado,0.0,6,15.0,La Zarzuela,1,0,0,0,0,0,0,0,0
2,3,OtherHorse,57,2,3,OtherOwner,J.M.OSORIO,J.GELABERT,0,[01],13 de noviembre de 2022,11:45,H - Blando,1500,Liso,C,0,Parcialmente nublado,0.0,6,15.0,La Zarzuela,1,0,0,0,0,0,0,0,0
3,4,OtherHorse,57,2,4,OtherOwner,B.RAMA,B. FAYOS,-8,"[08,15]",13 de noviembre de 2022,11:45,H - Blando,1500,Liso,C,0,Parcialmente nublado,0.0,6,15.0,La Zarzuela,0,0,0,0,0,0,0,0,1
4,5,OtherHorse,57,2,2,OtherOwner,OtherTrainer,R.N.VALLE,-8,"[09,11]",13 de noviembre de 2022,11:45,H - Blando,1500,Liso,C,0,Parcialmente nublado,0.0,6,15.0,La Zarzuela,0,0,0,0,0,0,0,0,1


## **Tratamiento (UltimasActuaciones)**


In [553]:
def count_lastRaces(arrayLast):
  index = 0
  for element in arrayLast:
    element = element.replace(',', '')
    element = element.replace('[', '')
    element = element[:len(element)-1]
    i = 0
    j = 0
    sum = 0
    if 'Debutante' in element:
      sum = 10
      element = []
    else:
      element = element.replace('NP', '08')
      element = element.replace('Desc', '08')
      element = element.replace(' ', '')
      while(i < len(element)):
        if(i == 0 and len(element)>4):
          j = j+2
          sum += int(element[i:j])**1.3
          i = i+2 
        elif (i == 2 and len(element)>4):
          j = j+2
          sum += int(element[i:j])**1.2
          i = i+2 
        else:
          j = j+2
          print(index)
          sum += int(element[i:j])
          i = i+2
      print('******INDEX ', index+2, '*****')
      print('Sum -->', sum, '/Len -->', len(element))
      sum = sum / len(element)
      print('Result -->', sum, '/Element -->' ,element)
    train.at[index, 'MediaUltimasActuaciones'] = sum
    train.at[index, 'CantidadActuaciones'] = (len(element)/2)
    index += 1

      



t_array = train["UltimasActuaciones"]
count_lastRaces(t_array)

0
******INDEX  2 *****
Sum --> 8.740320469781409 /Len --> 6
Result --> 1.4567200782969014 /Element --> 020401
1
1
******INDEX  3 *****
Sum --> 10 /Len --> 4
Result --> 2.5 /Element --> 0406
2
******INDEX  4 *****
Sum --> 1 /Len --> 2
Result --> 0.5 /Element --> 01
3
3
******INDEX  5 *****
Sum --> 23 /Len --> 4
Result --> 5.75 /Element --> 0815
4
4
******INDEX  6 *****
Sum --> 20 /Len --> 4
Result --> 5.0 /Element --> 0911
5
5
******INDEX  7 *****
Sum --> 12 /Len --> 4
Result --> 3.0 /Element --> 0705
6
6
******INDEX  8 *****
Sum --> 9 /Len --> 4
Result --> 2.25 /Element --> 0108
7
******INDEX  9 *****
Sum --> 2 /Len --> 2
Result --> 1.0 /Element --> 02
8
******INDEX  10 *****
Sum --> 17.38131462655539 /Len --> 6
Result --> 2.896885771092565 /Element --> 050404
9
9
9
******INDEX  11 *****
Sum --> 44.28258458066108 /Len --> 10
Result --> 4.428258458066108 /Element --> 0714050102
10
10
******INDEX  12 *****
Sum --> 10 /Len --> 4
Result --> 2.5 /Element --> 0307
11
11
11
******INDEX  13 **

In [554]:
 train['MediaUltimasActuaciones'].head(20)

0     1.456720
1     2.500000
2     0.500000
3     5.750000
4     5.000000
5     3.000000
6     2.250000
7     1.000000
8     2.896886
9     4.428258
10    2.500000
11    2.900193
12    2.500000
13    6.000000
14    3.629729
15    1.500000
16    2.000000
17    2.000000
18    2.000000
19    2.574867
Name: MediaUltimasActuaciones, dtype: float64

## **Tratamiento (Fecha)**


In [555]:
import datetime as dt
from datetime import date


def switchMonth(month):
  if month == 'ENERO':  
   return '01'
  elif month == 'FEBRERO':
    return '02'            
  elif month == 'MARZO':
   return '03'
  elif month == 'ABRIL':
    return '04' 
  elif month == 'MAYO':
    return '05'                  
  elif month == 'JUNIO':
    return '06'
  elif month == 'JULIO':
    return '07'
  elif month == 'AGOSTO':
    return '08'
  elif month == 'SEPTIEMBRE':
    return '09'
  elif month == 'OCTUBRE':
     return '10'
  elif month == 'NOVIEMBRE':
    return '11'
  elif month == 'DICIEMBRE':
    return '12'

def convertDate(fechas):
  index = 0
  for element in fechas:
    sum = 0
    year = element[-4:]
    dateAux = year
    day = element[:2]
    if(int(day) < 10):
      day = "0" + day[:1]
    month = element[5:]
    month = ''.join(month.split())[:-6].upper()
    month = switchMonth(month)
    dateAux = dateAux + "-" + month + "-" + day

    today = date.today()
    yearToday = int(str(today)[:4])
    monthToday = int(str(today)[5:7])
    dayToday = int(str(today)[8:10])
    if(yearToday > int(year)):
      sum += (yearToday - int(year))*365
    sum += (monthToday - int(month))*30
    sum += (dayToday - int(day))
    datetimeAux = dt.datetime(int(year), int(month), int(day))
    train.at[index, 'FechaAux'] = datetimeAux

    train.at[index, 'year'] = int(year)
    train.at[index, 'month'] = int(month)
    train.at[index, 'day'] = int(day)
    calculateSeason(int(month), index)

    train.at[index, 'DiasDesdeCarrera'] = sum
    index += 1
   
def calculateSeason(month, index):
  train['Invierno'] = 0
  train['Primavera'] = 0
  train['Verano'] = 0
  train['Otoño'] = 0
  if(month >= 12 and month <=2):
     train.at[index, 'Invierno'] = month
  if(month >= 3 and month <=5):
     train.at[index, 'Primavera'] = month
  if(month >= 6 and month <=8):
     train.at[index, 'Verano'] = month
  if(month >= 9 and month <=11):
     train.at[index, 'Otoño'] = month




t_array = train["Fecha"] 
convertDate(t_array)
train['FechaAux'].head(120)

0     2022-11-13
1     2022-11-13
2     2022-11-13
3     2022-11-13
4     2022-11-13
         ...    
115   2022-11-06
116   2022-11-06
117   2022-11-06
118   2022-11-06
119   2022-11-06
Name: FechaAux, Length: 120, dtype: datetime64[ns]

In [556]:
train.head(5)

,Puesto,NombreCaballo,Peso,Edad,Mantilla,Propietario,Preparador,Jinete,Problemas,UltimasActuaciones,Fecha,Hora,Terreno,Distancia,Tipo,Categoría,SentidoHipodromo,Meteorología,LLuvia,Viento,Temperatura,Hipodromo,Problema_Nulo,Problema_1,Problema_2,Problema_3,Problema_4,Problema_5,Problema_6,Problema_7,Problema_8,MediaUltimasActuaciones,CantidadActuaciones,FechaAux,year,month,day,Invierno,Primavera,Verano,Otoño,DiasDesdeCarrera
0,1,OtherHorse,57,2,6,OtherOwner,OtherTrainer,OtherJockey,0,"[02,04,01]",13 de noviembre de 2022,11:45,H - Blando,1500,Liso,C,0,Parcialmente nublado,0.0,6,15.0,La Zarzuela,1,0,0,0,0,0,0,0,0,1.45672,3.0,2022-11-13,2022.0,11.0,13.0,0,0,0,0,24.0
1,2,OtherHorse,57,2,5,OtherOwner,G.ARIZKORRETA,V. JANACEK,0,"[04,06]",13 de noviembre de 2022,11:45,H - Blando,1500,Liso,C,0,Parcialmente nublado,0.0,6,15.0,La Zarzuela,1,0,0,0,0,0,0,0,0,2.50000,2.0,2022-11-13,2022.0,11.0,13.0,0,0,0,0,24.0
2,3,OtherHorse,57,2,3,OtherOwner,J.M.OSORIO,J.GELABERT,0,[01],13 de noviembre de 2022,11:45,H - Blando,1500,Liso,C,0,Parcialmente nublado,0.0,6,15.0,La Zarzuela,1,0,0,0,0,0,0,0,0,0.50000,1.0,2022-11-13,2022.0,11.0,13.0,0,0,0,0,24.0
3,4,OtherHorse,57,2,4,OtherOwner,B.RAMA,B. FAYOS,-8,"[08,15]",13 de noviembre de 2022,11:45,H - Blando,1500,Liso,C,0,Parcialmente nublado,0.0,6,15.0,La Zarzuela,0,0,0,0,0,0,0,0,1,5.75000,2.0,2022-11-13,2022.0,11.0,13.0,0,0,0,0,24.0
4,5,OtherHorse,57,2,2,OtherOwner,OtherTrainer,R.N.VALLE,-8,"[09,11]",13 de noviembre de 2022,11:45,H - Blando,1500,Liso,C,0,Parcialmente nublado,0.0,6,15.0,La Zarzuela,0,0,0,0,0,0,0,0,1,5.00000,2.0,2022-11-13,2022.0,11.0,13.0,0,0,0,0,24.0


In [557]:
import datetime

def daysSincePreviousRace(nombres):
  index = 0
  for caballo in nombres:
    indexNombre = 0
    nombresAparece = []
    for otroCaballo in nombres:
      if(otroCaballo == caballo):
        nombresAparece.append(indexNombre)
      indexNombre += 1

    day_actual = train.at[index, 'day']
    month_actual = train.at[index, 'month']
    year_actual = train.at[index, 'year']

    fechaCaballoActual = dt.datetime(int(year_actual), int(month_actual), int(day_actual))
    fechaMasReciente = dt.datetime(2020, 1, 1)
    indexAux2 = 0
    fechaCambiada = False
    for aparicion in nombresAparece:
      if(train.at[aparicion, 'FechaAux'] < fechaCaballoActual): 
        if(indexAux2 == 0):
          fechaMasReciente = train.at[aparicion, 'FechaAux']
          fechaCambiada = True
          indexAux2 += 1
        elif(train.at[aparicion, 'FechaAux'] > fechaMasReciente):
          fechaMasReciente =  train.at[aparicion, 'FechaAux']
    if(fechaCambiada):
      diferenciaDias = (fechaCaballoActual - fechaMasReciente).days
    else:
      diferenciaDias = 45 # Penalización si no aparece en carrera anterior
    train.at[index, 'DaysSincePreviousRace'] = diferenciaDias
    index += 1


eventos = train['NombreCaballo']
daysSincePreviousRace(eventos)
train['DaysSincePreviousRace'].head(35)
# Recorro los nombres de caballos y guardo en un array los index de las columnas ligadas a un nombre de Caballo y despues recorrer las fechas y cuando coincida el array comprobar si ese dato
# es mas antiguo que la fecha de la fila actual y en el caso de serlo compruebo si es la más antigua de entre las anteriores participaciones











0     7.0
1     7.0
2     7.0
3     7.0
4     7.0
5     7.0
6     7.0
7     7.0
8     7.0
9     7.0
10    7.0
11    7.0
12    7.0
13    7.0
14    7.0
15    7.0
16    7.0
17    7.0
18    7.0
19    7.0
20    7.0
21    7.0
22    7.0
23    7.0
24    7.0
25    7.0
26    7.0
27    7.0
28    7.0
29    7.0
30    7.0
31    7.0
32    7.0
33    7.0
34    7.0
Name: DaysSincePreviousRace, dtype: float64

## **Tratamiento (Hora)**


In [558]:
train['Hora'] = train['Hora'].str.replace(':', '') 

## **Tratamiento (Terreno)**


In [559]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 43 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Puesto                   501 non-null    int64         
 1   NombreCaballo            501 non-null    object        
 2   Peso                     501 non-null    object        
 3   Edad                     501 non-null    int64         
 4   Mantilla                 501 non-null    int64         
 5   Propietario              501 non-null    object        
 6   Preparador               501 non-null    object        
 7   Jinete                   501 non-null    object        
 8   Problemas                501 non-null    object        
 9   UltimasActuaciones       501 non-null    object        
 10  Fecha                    501 non-null    object        
 11  Hora                     501 non-null    object        
 12  Terreno                  501 non-nul

## **Tratamiento (Distancia)**


## **Tratamiento (Tipo)**


## **Tratamiento (Categoria)**


## **Tratamiento (SentidoHipodromo)**


## **Tratamiento (Meteorologia)**


## **Tratamiento (Lluvia)**


## **Tratamiento (Viento)**


## **Tratamiento (TemperaturaMax)**


## **Tratamiento (TemperaturaMin)**


## **Tratamiento (Hipodromo)**


# Preprocesamiento Extra
---

## Calculo Contricantes
---

In [560]:
def calculoContrincantes(listaF, listaH, longitud):
  for fecha in listaF:
    for horario in listaH:
      index = 0
      listaIndex = []
      sum = 0
      while(index < longitud):
        if((train.at[index, 'Fecha'] == fecha ) and (train.at[index, 'Hora'] == horario )):
          listaIndex.append(index)
          sum += 1
        index += 1
      for indice in listaIndex:
        train.at[indice, 'Contrincantes'] = sum


listaFechas = train['Fecha'].unique().tolist()
listaHorarios = train['Hora'].unique().tolist()
longitud = len(train.index)

calculoContrincantes(listaFechas, listaHorarios, longitud)




## Busqueda Caballos Ganadores en ciertas condiciones
---

### Busqueda Distancia
---

In [562]:
def comprobacionTipoDistancia(indice):
  tipo = 0   # 0, 1, 2 / Corta, Media, Larga
  val = train.at[indice, 'Distancia']
  if(val >= 3200):
    tipo = 2
  if((val >= 1600) & (val <= 2400)):
    tipo = 1
  return tipo;

def destrezaDistancia(indices, fechas):                      
  for i in range(indices):
    tipo = comprobacionTipoDistancia(i)
    fecha = train.at[i, 'FechaAux']
    media = 0 # Media carreras
    sum = 0 # Numero Carreras
    index = 0
    mediaAux = 0 
    sumAux = 0
    for j in range(indices):
      fechaAux = train.at[j, 'FechaAux']
      tipoAux = comprobacionTipoDistancia(j)
      if((fechaAux < fecha) & (tipoAux == tipo)):
        contrincante = train.at[index, 'Contrincantes']
        media += (train.at[index, 'Puesto'] / contrincante)
        sum += 1
      elif((fechaAux < fecha) & (tipoAux != tipo)):
        contrincante = train.at[index, 'Contrincantes']
        mediaAux += (train.at[index, 'Puesto'] / contrincante)
        sumAux += 1
      index += 1
    if(sum != 0):
      media /= sum
    if(sumAux != 0):
      mediaAux /= sumAux
    if((sum > 2) & (((1.5* media) < mediaAux))): # Si hay mas de 2 participaciones en esas condiciones y con una media al menos un 33% mas baja , es valido
      train.at[i, 'DestrezaDistancia'] = 1
    else:
      train.at[i, 'DestrezaDistancia'] = 0
        

indices = len(train.index)
fechas = train['FechaAux'].tolist()
destrezaDistancia(indices, fechas)


#Len Intervalos
print(len(train[(train['Distancia'] < 1600).tolist()]))
print(len(train[((train['Distancia'] >= 1600) & (train['Distancia'] <= 2400) ).tolist()]))
print(len(train[(train['Distancia'] >= 3200).tolist()]))

200
292
9


# Visualizacion

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(data=train, x='Distancia', y='Puesto')

plt.show()

In [ ]:
print(len(train[(train['Distancia'] < 1600).tolist()]))

print(len(train[((train['Distancia'] >= 1600) & (train['Distancia'] <= 2400) ).tolist()]))

print(len(train[(train['Distancia'] >= 3200).tolist()]))

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(data=train, x='Hora', y='Puesto')

plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(data=train, x='Categoría', y='Puesto')

plt.show()

In [ ]:
plt.figure(figsize=(50,8))
sns.boxplot(data=train, x='Tipo', y='Puesto')

plt.show()

In [ ]:
plt.figure(figsize=(40,8))
sns.boxplot(data=train, x='Meteorología', y='Puesto')

plt.show()

In [ ]:
plt.figure(figsize=(40,8))
sns.boxplot(data=train, x='LLuvia', y='Puesto')

plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(data=train, x='Viento', y='Puesto')

plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(data=train, x='Temperatura', y='Puesto')

plt.show()

# Preprocesamiento Final

In [ ]:
train.info()

In [ ]:
import sklearn
import sklearn.compose
#Solo ejecutar previo a esto product para separar las palabras
column_transformer_train = sklearn.compose.ColumnTransformer(transformers=[
    ("drop", "drop", ['UltimasActuaciones','Fecha','year','day', 'Problemas','Hora', 'FechaAux', 'Mantilla', 'Terreno', 'Tipo','Categoría', 'Meteorología', 'LLuvia', 'Viento', 'Temperatura', 'Hipodromo', 'Contrincantes', 'DestrezaDistancia']),
    ("scale", sklearn.preprocessing.StandardScaler(), []),
    # ("num", sklearn.preprocessing.MinMaxScaler(), ['Distancia', 'Edad', 'Peso']),
    ("one-hot", sklearn.preprocessing.OneHotEncoder(handle_unknown="ignore"), ['NombreCaballo','Propietario', 'Preparador', 'Jinete']),
], remainder='passthrough');

# column_transformer = sklearn.compose.ColumnTransformer(transformers=[
#     ("drop", "drop", ['UltimasActuaciones','Fecha','year','day', 'Problemas','Hora']),
#     ("scale", sklearn.preprocessing.StandardScaler(), []),
#     ("num", sklearn.preprocessing.MinMaxScaler(), ['Distancia', 'Edad', 'Peso']),
#     ("one-hot", sklearn.preprocessing.OneHotEncoder(handle_unknown="ignore"), ['NombreCaballo', 'Mantilla','Propietario', 'Preparador', 'Jinete', 'FechaAux', 'Terreno', 'Tipo','Categoría', 'Meteorología', 'LLuvia', 'Viento', 'TemperaturaMax', 'TemperaturaMin', 'Hipodromo']),
# ], remainder='passthrough');

In [ ]:
train = train.reset_index()
X=train
X = X.drop(["Puesto"], axis = 1)

In [ ]:
X_transform = column_transformer_train.fit_transform(X)

In [ ]:
column_transformer_train.get_feature_names()

In [ ]:
train 
X_transform
X = X_transform       #Nos quedamos con todo menos la columna objetivo, en este caso 'price'
y = train["Puesto"]

In [ ]:
train[['NombreCaballo','MediaUltimasActuaciones']].head(128)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1920)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import SGDRegressor

#Regresión lineal
linear = LinearRegression()
linear.fit(X_train, y_train)

#Regresión de Ridge

ridge = Ridge()
ridge.fit(X_train, y_train)

#LASSO

lasso = linear_model.Lasso()
lasso.fit(X_train, y_train)

#ElasticNet

elasticnet = ElasticNet()
elasticnet.fit(X_train, y_train);


reg = SGDRegressor(max_iter=50, random_state=43)
reg.fit(X_train, y_train);

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error
import xgboost as xgb

for i in range(0, 11):
  for j in range(0, 11):
    #     xgb_model = xgb.XGBRegressor(objective = 'reg:squarederror', reg_alpha=i*0.1, reg_lambda=j*0.1, verbosity=0, random_state=42, eta=0.2, tree_method="exact") 148

    elasticnet = ElasticNet(alpha=i*0.1, l1_ratio=j*0.1)
    elasticnet.fit(X_train, y_train)
    print('ElasticNet (train): ' + str(mean_absolute_error(y_train, elasticnet.predict(X_train))))
    print(f'{i} {j} ' + str(mean_absolute_error(y_test, elasticnet.predict(X_test))))

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error
import xgboost as xgb

for i in range(0, 11):
  for j in range(0, 11):
    #     xgb_model = xgb.XGBRegressor(objective = 'reg:squarederror', reg_alpha=i*0.1, reg_lambda=j*0.1, verbosity=0, random_state=42, eta=0.2, tree_method="exact") 148

    xgb_model = xgb.XGBRegressor(objective = 'reg:squarederror', reg_alpha=i*0.1, reg_lambda=j*0.1, verbosity=0, random_state=42, eta=0.2, tree_method="exact") #subsample ojo
    xgb_model.fit(X_train, y_train)
    print('ElasticNet (train): ' + str(mean_absolute_error(y_train, xgb_model.predict(X_train))))
    print(f'{i} {j} ' + str(mean_absolute_error(y_test, xgb_model.predict(X_test))))

In [ ]:
import xgboost as xgb

xgb_model = xgb.XGBRegressor(objective = 'reg:squarederror', reg_alpha=7*0.1, reg_lambda=4*0.1, verbosity=0, random_state=42, eta=0.2, tree_method="exact")

xgb_model.fit(X, y)
#print('Lineal (train): ' + str(mean_absolute_error(y_train, xgb_model.predict(X))))
#print('Lineal (test): ' + str(mean_absolute_error(y_test, xgb_model.predict(X_test))))

In [ ]:
!pip install catboost
import catboost as cb

In [ ]:
params = {"depth": 10, "learning_rate": 0.5, "iterations": 500, "l2_leaf_reg": 5} #Iterations CatBoost
model_cat_tun = cb.CatBoostClassifier(**params)
model_cat_tun.fit(X,y)

In [ ]:
from sklearn.metrics import mean_absolute_error

print('Lineal (train): ' + str(mean_absolute_error(y_train, linear.predict(X_train))))
print('Lineal (test): ' + str(mean_absolute_error(y_test, linear.predict(X_test))))

print('---')

print('Ridge (train): ' + str(mean_absolute_error(y_train, ridge.predict(X_train))))
print('Ridge (test): ' + str(mean_absolute_error(y_test, ridge.predict(X_test))))

print('---')

print('LASSO (train): ' + str(mean_absolute_error(y_train, lasso.predict(X_train))))
print('LASSO (test): ' + str(mean_absolute_error(y_test, lasso.predict(X_test))))

print('---')

print('ElasticNet (train): ' + str(mean_absolute_error(y_train, elasticnet.predict(X_train))))
print('ElasticNet (test): ' + str(mean_absolute_error(y_test, elasticnet.predict(X_test))))

print('---')

print('SGD (train): ' + str(mean_absolute_error(y_train, reg.predict(X_train))))
print('SGD (test): ' + str(mean_absolute_error(y_test, reg.predict(X_test))))

In [ ]:
from sklearn.metrics import accuracy_score

print('Lineal (train): ' + str(linear.score(X_train, y_train)))
print('Lineal (test): ' + str(linear.score(X_test, y_test)))
print('---')
print('Ridge (train): ' + str(ridge.score(X_train, y_train)))
print('Ridge (test): ' + str(ridge.score(X_test, y_test)))
print('---')
print('LASSO (train): ' + str(lasso.score(X_train, y_train)))
print('LASSO (test): ' + str(lasso.score(X_test, y_test)))
print('---')
print('ElasticNet (train): ' + str(elasticnet.score(X_train, y_train)))
print('ElasticNet (test): ' + str(elasticnet.score(X_test, y_test)))

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
print('Lineal (train): ' + str(sqrt(mean_squared_error(y_train, linear.predict(X_train)))))
print('Lineal (test): ' + str(sqrt(mean_squared_error(y_test, linear.predict(X_test)))))
print('---')
print('Ridge (train): ' + str(sqrt(mean_squared_error(y_train, ridge.predict(X_train)))))
print('Ridge (test): ' + str(sqrt(mean_squared_error(y_test, ridge.predict(X_test)))))
print('---')
print('LASSO (train): ' + str(sqrt(mean_squared_error(y_train, lasso.predict(X_train)))))
print('LASSO (test): ' + str(sqrt(mean_squared_error(y_test, lasso.predict(X_test)))))
print('---')
print('ElasticNet (train): ' + str(sqrt(mean_squared_error(y_train, elasticnet.predict(X_train)))))
print('ElasticNet (test): ' + str(sqrt(mean_squared_error(y_test, elasticnet.predict(X_test)))))

# Optimización
---

Linear
---

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
fit_intercept = [True, False]
copy_x = [True, False]
normalize = [True, False]

In [ ]:
linear_Grid = GridSearchCV(
  estimator = LinearRegression(),
  param_grid = dict(fit_intercept = fit_intercept, copy_X = copy_x, normalize = normalize),
  cv = 5
)
linear_Grid.fit(X, y)

In [ ]:
linear_Grid.best_estimator_

In [ ]:
linear_Grid.best_params_

In [ ]:
linear_Grid.best_score_

In [ ]:
linear = LinearRegression(fit_intercept=True, normalize=False, copy_X=True)
linear.fit(X_train, y_train)

In [ ]:
print("MAE: ")
print('Train: ' + str(mean_absolute_error(y_train, linear.predict(X_train))))
print('Test: ' + str(mean_absolute_error(y_test, linear.predict(X_test))))
print("---------------------------------------")
print("R^2: ")
print("Train: " + str(linear.score(X_train, y_train)))
print("Test: " + str(linear.score(X_test, y_test)))
print("---------------------------------------")
print("RMST: ")
print('Train: ' + str(sqrt(mean_squared_error(y_train, linear.predict(X_train)))))
print('Test: ' + str(sqrt(mean_squared_error(y_train, linear.predict(X_train)))))

Se puede observar que no mejora ya que los parámetros que nos da el GridSearch son los mismos que tiene de base.

Ridge
---

In [ ]:
ridge_Grid = GridSearchCV(
  estimator = Ridge(),
  param_grid = dict(alpha= np.linspace(0, 1, 50), copy_X = copy_x, fit_intercept=fit_intercept, normalize = normalize),
  cv = 5

)
ridge_Grid.fit(X, y)

In [ ]:
ridge_Grid.best_estimator_

In [ ]:
ridge_Grid.best_params_

In [ ]:
ridge_Grid.best_score_

In [ ]:
ridge = Ridge(alpha=0.32653061224489793, copy_X=True, fit_intercept=True,
      normalize=True, random_state=1337, solver='auto', tol=0.001)
ridge.fit(X_train, y_train)

In [ ]:
print("MAE: ")
print('Train: ' + str(mean_absolute_error(y_train, ridge.predict(X_train))))
print('Test: ' + str(mean_absolute_error(y_test, ridge.predict(X_test))))
print("---------------------------------------")
print("R^2: ")
print("Train: " + str(ridge.score(X_train, y_train)))
print("Test: " + str(ridge.score(X_test, y_test)))
print("---------------------------------------")
print("RMST: ")
print('Train: ' + str(sqrt(mean_squared_error(y_train, ridge.predict(X_train)))))
print('Test: ' + str(sqrt(mean_squared_error(y_test, ridge.predict(X_test)))))

Parece que empeora ligeramente con respecto a los valores por defecto que tenía antes.

LASSO
---

In [ ]:
lasso_Grid = GridSearchCV(estimator = linear_model.Lasso(), param_grid = dict(alpha= np.linspace(0.05,1,20)), cv = 5)
lasso_Grid.fit(X, y)

In [ ]:
lasso_Grid.best_params_

In [ ]:
lasso = linear_model.Lasso(alpha=1, random_state=1337)
lasso.fit(X_train, y_train)

In [ ]:
print("MAE: ")
print('Train: ' + str(mean_absolute_error(y_train, lasso.predict(X_train))))
print('Test: ' + str(mean_absolute_error(y_test, lasso.predict(X_test))))
print("---------------------------------------")
print("R^2: ")
print("Train: " + str(lasso.score(X_train, y_train)))
print("Test: " + str(lasso.score(X_test, y_test)))
print("---------------------------------------")
print("RMST: ")
print('Train: ' + str(sqrt(mean_squared_error(y_train, lasso.predict(X_train)))))
print('Test: ' + str(sqrt(mean_squared_error(y_train, lasso.predict(X_train)))))

Se queda igual que con los valores de base.

ElasticNet
---

Este regresor a veces funciona y a veces explota, dejamos guardado los resultados de cuando ha funcionado. De todos modos nos sale el siguiente error:<br>


```
# /usr/local/lib/python3.6/dist-packages/sklearn linear_model_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge.
```



In [ ]:
elastic_Grid = GridSearchCV(
  estimator = ElasticNet(),
  param_grid = dict(alpha=np.linspace(0.1,1,2), l1_ratio=np.linspace(0,1,2)),
  cv = 5
)
elastic_Grid.fit(X, y)

In [ ]:
elastic_Grid.best_params_

In [ ]:
elasticnet = ElasticNet(alpha = 1.0, l1_ratio = 1, fit_intercept=True, normalize=False, copy_X=True)
elasticnet.fit(X_train, y_train)

In [ ]:
print("MAE: ")
print('Train: ' + str(mean_absolute_error(y_train, elasticnet.predict(X_train))))
print('Test: ' + str(mean_absolute_error(y_test, elasticnet.predict(X_test))))
print("---------------------------------------")
print("R^2: ")
print("Train: " + str(lasso.score(X_train, y_train)))
print("Test: " + str(lasso.score(X_test, y_test)))
print("---------------------------------------")
print("RMST: ")
print('Train: ' + str(sqrt(mean_squared_error(y_train, elasticnet.predict(X_train)))))
print('Test: ' + str(sqrt(mean_squared_error(y_train, elasticnet.predict(X_train)))))

# Validación Cruzada
---

## Linear

MAE

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_validate

In [ ]:
cv_scores_linearMAE = cross_val_score(
                estimator = linear,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_absolute_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_linearMAE}")
print(f"Média métricas de validación cruzada: {cv_scores_linearMAE.mean()}")

R^2

In [ ]:
cv_scores_linearR = cross_val_score(
                estimator = linear,
                X         = X,
                y         = y,
                scoring   = 'r2',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_linearR}")
print(f"Média métricas de validación cruzada: {cv_scores_linearR.mean()}")

RMSE

In [ ]:
cv_scores_linearRMSE = cross_val_score(
                estimator = linear,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_squared_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_linearRMSE}")
print(f"Média métricas de validación cruzada: {cv_scores_linearRMSE.mean()}")

## Ridge
---

MAE

In [ ]:
cv_scores_ridgeMAE = cross_val_score(
                estimator = ridge,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_absolute_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_ridgeMAE}")
print(f"Média métricas de validación cruzada: {cv_scores_ridgeMAE.mean()}")

R^2

In [ ]:
cv_scores_ridgeR = cross_val_score(
                estimator = ridge,
                X         = X,
                y         = y,
                scoring   = 'r2',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_ridgeR}")
print(f"Média métricas de validación cruzada: {cv_scores_ridgeR.mean()}")

RMSE

In [ ]:
cv_scores_ridgeRMSE = cross_val_score(
                estimator = ridge,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_squared_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_ridgeRMSE}")
print(f"Média métricas de validación cruzada: {cv_scores_ridgeRMSE.mean()}")

## LASSO
---

MAE

In [ ]:
cv_scores_lassoMAE = cross_val_score(
                estimator = lasso,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_absolute_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_lassoMAE}")
print(f"Média métricas de validación cruzada: {cv_scores_lassoMAE.mean()}")

R^2

In [ ]:
cv_scores_lassoR = cross_val_score(
                estimator = lasso,
                X         = X,
                y         = y,
                scoring   = 'r2',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_lassoR}")
print(f"Média métricas de validación cruzada: {cv_scores_lassoR.mean()}")

RMSE

In [ ]:
cv_scores_lassoRMSE = cross_val_score(
                estimator = lasso,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_squared_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_lassoRMSE}")
print(f"Média métricas de validación cruzada: {cv_scores_lassoRMSE.mean()}")

## ElasticNet
---

MAE

In [ ]:
cv_scores_netMAE = cross_val_score(
                estimator = elasticnet,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_absolute_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_netMAE}")
print(f"Média métricas de validación cruzada: {cv_scores_netMAE.mean()}")

R^2

In [ ]:
cv_scores_netR = cross_val_score(
                estimator = elasticnet,
                X         = X,
                y         = y,
                scoring   = 'r2',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_netR}")
print(f"Média métricas de validación cruzada: {cv_scores_netR.mean()}")

RMSE

In [ ]:
cv_scores_netRMSE = cross_val_score(
                estimator = elasticnet,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_squared_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_netRMSE}")
print(f"Média métricas de validación cruzada: {cv_scores_netRMSE.mean()}")

## XGBoost
---

MAE

In [ ]:
cv_scores_netMAE = cross_val_score(
                estimator = xgb_model,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_absolute_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_netMAE}")
print(f"Média métricas de validación cruzada: {cv_scores_netMAE.mean()}")

R^2

In [ ]:
cv_scores_netR = cross_val_score(
                estimator = xgb_model,
                X         = X,
                y         = y,
                scoring   = 'r2',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_netR}")
print(f"Média métricas de validación cruzada: {cv_scores_netR.mean()}")

RMSE

In [ ]:
cv_scores_netRMSE = cross_val_score(
                estimator = xgb_model,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_squared_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_netRMSE}")
print(f"Média métricas de validación cruzada: {cv_scores_netRMSE.mean()}")

# Prueba
---

In [ ]:
# train = train.drop(["index"], axis = 1)
train = test

# Ajuste Preprocesamiento
- Borrar Puesto
- Ajuste Listas Apariciones

In [ ]:
train['NombreCaballo'] = train['NombreCaballo'].str.partition('(')[0]
print(listAparicionesHorse)

listTestHorse= eraseBlank(listaNombreCaballo)

train.loc[~train["Propietario"].isin(listAparicionesOwner), "Propietario"] = "OtherOwner"
train.loc[~train["NombreCaballo"].isin(listAparicionesHorse), "NombreCaballo"] = "OtherHorse"
train.loc[~train["Jinete"].isin(listAparicionesJockey), "Jinete"] = "OtherJockey"
train.loc[~train["Preparador"].isin(listAparicionesTrainer), "Preparador"] = "OtherTrainer"

# Preprocesamiento
---

In [ ]:
train.head(5)

## **Tratamiento (Peso)**

In [ ]:
train['Peso'] = train['Peso'].str.partition('-')[0]
train['Peso'] = train['Peso'].str[:2]



In [ ]:
train['Peso'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

## **Tratamiento (Edad)**

In [ ]:
train['Edad'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

## **Tratamiento (Mantilla)**

In [ ]:
train['Mantilla'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

## **Tratamiento (Problemas)**


In [ ]:
train['Problemas'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

In [ ]:
train['Problemas'] = train['Problemas'].fillna('0')
train['Problema_Nulo'] = np.where(train['Problemas'].str.contains('0'),1,0)
train['Problema_1'] = np.where(train['Problemas'].str.contains('1'),1,0)
train['Problema_2'] = np.where(train['Problemas'].str.contains('2'),1,0)
train['Problema_3'] = np.where(train['Problemas'].str.contains('3'),1,0)
train['Problema_4'] = np.where(train['Problemas'].str.contains('4'),1,0)
train['Problema_5'] = np.where(train['Problemas'].str.contains('5'),1,0)
train['Problema_6'] = np.where(train['Problemas'].str.contains('6'),1,0)
train['Problema_7'] = np.where(train['Problemas'].str.contains('7'),1,0)
train['Problema_8'] = np.where(train['Problemas'].str.contains('8'),1,0)
train.head()



## **Tratamiento (UltimasActuaciones)**


In [ ]:
def count_lastRaces(arrayLast):
  index = 0
  for element in arrayLast:
    element = element.replace(',', '')
    element = element.replace('[', '')
    element = element[:len(element)-1]
    i = 0
    j = 0
    sum = 0
    if 'Debutante' in element:
      sum = 10
      element = []
    else:
      element = element.replace('NP', '08')
      element = element.replace('Desc', '08')
      element = element.replace(' ', '')
      while(i < len(element)):
        if(i == 0 and len(element)>4):
          j = j+2
          sum += int(element[i:j])**1.3
          i = i+2 
        elif (i == 2 and len(element)>4):
          j = j+2
          sum += int(element[i:j])**1.2
          i = i+2 
        else:
          j = j+2
          sum += int(element[i:j])
          i = i+2
      sum = sum / len(element)
      if(index < 10):
        print('******INDEX ', index+2, '*****')
        print('Sum -->', sum, '/Len -->', len(element))
        print('Result -->', sum, '/Element -->' ,element)
    train.at[index, 'MediaUltimasActuaciones'] = sum
    train.at[index, 'CantidadActuaciones'] = len(element)
    index += 1

      



t_array = train["UltimasActuaciones"]
count_lastRaces(t_array)

In [ ]:
 train['MediaUltimasActuaciones'].head(20)

## **Tratamiento (Fecha)**


In [ ]:
import datetime as dt
from datetime import date


def switchMonth(month):
  if month == 'ENERO':  
   return '01'
  elif month == 'FEBRERO':
    return '02'            
  elif month == 'MARZO':
   return '03'
  elif month == 'ABRIL':
    return '04' 
  elif month == 'MAYO':
    return '05'                  
  elif month == 'JUNIO':
    return '06'
  elif month == 'JULIO':
    return '07'
  elif month == 'AGOSTO':
    return '08'
  elif month == 'SEPTIEMBRE':
    return '09'
  elif month == 'OCTUBRE':
     return '10'
  elif month == 'NOVIEMBRE':
    return '11'
  elif month == 'DICIEMBRE':
    return '12'

def convertDate(fechas):
  index = 0
  for element in fechas:
    sum = 0
    year = element[-4:]
    dateAux = year
    day = element[:2]
    if(int(day) < 10):
      day = "0" + day[:1]
    month = element[5:]
    month = ''.join(month.split())[:-6].upper()
    month = switchMonth(month)
    dateAux = dateAux + "-" + month + "-" + day

    today = date.today()
    yearToday = int(str(today)[:4])
    monthToday = int(str(today)[5:7])
    dayToday = int(str(today)[8:10])
    if(yearToday > int(year)):
      sum += (yearToday - int(year))*365
    sum += (monthToday - int(month))*30
    sum += (dayToday - int(day))
    datetimeAux = dt.datetime(int(year), int(month), int(day))
    train.at[index, 'FechaAux'] = datetimeAux

    train.at[index, 'year'] = int(year)
    train.at[index, 'month'] = int(month)
    train.at[index, 'day'] = int(day)
    calculateSeason(int(month), index)

    train.at[index, 'DiasDesdeCarrera'] = sum
    index += 1
   
def calculateSeason(month, index):
  train['Invierno'] = 0
  train['Primavera'] = 0
  train['Verano'] = 0
  train['Otoño'] = 0
  if(month >= 12 and month <=2):
     train.at[index, 'Invierno'] = month
  if(month >= 3 and month <=5):
     train.at[index, 'Primavera'] = month
  if(month >= 6 and month <=8):
     train.at[index, 'Verano'] = month
  if(month >= 9 and month <=11):
     train.at[index, 'Otoño'] = month




t_array = train["Fecha"] 
convertDate(t_array)
train['FechaAux'].head(70)

In [ ]:
train.head(5)

In [ ]:
import datetime

def daysSincePreviousRace(nombres):
  index = 0
  for caballo in nombres:
    indexNombre = 0
    nombresAparece = []
    for otroCaballo in nombres:
      if(otroCaballo == caballo):
        nombresAparece.append(indexNombre)
      indexNombre += 1

    day_actual = train.at[index, 'day']
    month_actual = train.at[index, 'month']
    year_actual = train.at[index, 'year']

    fechaCaballoActual = dt.datetime(int(year_actual), int(month_actual), int(day_actual))
    fechaMasReciente = dt.datetime(2020, 1, 1)
    indexAux2 = 0
    fechaCambiada = False
    for aparicion in nombresAparece:
      if(train.at[aparicion, 'FechaAux'] < fechaCaballoActual): 
        if(indexAux2 == 0):
          fechaMasReciente = train.at[aparicion, 'FechaAux']
          fechaCambiada = True
          indexAux2 += 1
        elif(train.at[aparicion, 'FechaAux'] > fechaMasReciente):
          fechaMasReciente =  train.at[aparicion, 'FechaAux']
    if(fechaCambiada):
      diferenciaDias = (fechaCaballoActual - fechaMasReciente).days
    else:
      diferenciaDias = 45 # Penalización si no aparece en carrera anterior
    train.at[index, 'DaysSincePreviousRace'] = diferenciaDias
    index += 1


eventos = train['NombreCaballo']
daysSincePreviousRace(eventos)
train['DaysSincePreviousRace'].head(35)
# Recorro los nombres de caballos y guardo en un array los index de las columnas ligadas a un nombre de Caballo y despues recorrer las fechas y cuando coincida el array comprobar si ese dato
# es mas antiguo que la fecha de la fila actual y en el caso de serlo compruebo si es la más antigua de entre las anteriores participaciones











## **Tratamiento (Hora)**


In [ ]:
train['Hora'] = train['Hora'].str.replace(':', '') 

## **Tratamiento (Terreno)**


## **Tratamiento (Distancia)**


## **Tratamiento (Tipo)**


## **Tratamiento (Categoria)**


## **Tratamiento (SentidoHipodromo)**


## **Tratamiento (Meteorologia)**


## **Tratamiento (Lluvia)**


## **Tratamiento (Viento)**


## **Tratamiento (TemperaturaMax)**


## **Tratamiento (TemperaturaMin)**


## **Tratamiento (Hipodromo)**


# Preprocesamiento Extra
---

## Calculo Contricantes
---

In [ ]:
def calculoContrincantes(listaF, listaH, longitud):
  for fecha in listaF:
    for horario in listaH:
      index = 0
      listaIndex = []
      sum = 0
      while(index < longitud):
        if((train.at[index, 'Fecha'] == fecha ) and (train.at[index, 'Hora'] == horario )):
          listaIndex.append(index)
          sum += 1
        index += 1
      for indice in listaIndex:
        train.at[indice, 'Contrincantes'] = sum


listaFechas = train['Fecha'].unique().tolist()
listaHorarios = train['Hora'].unique().tolist()
longitud = len(train.index)

calculoContrincantes(listaFechas, listaHorarios, longitud)




## Busqueda Caballos Ganadores en ciertas condiciones

> Bloque con sangría


---

#Continuacion

In [ ]:
X_val = train
# X_val = X_val.reset_index()
ids = train
X_val = X_val.reset_index()
X_val.info()

In [ ]:
column_transformer_test = sklearn.compose.ColumnTransformer(transformers=[
 ("drop", "drop", ['UltimasActuaciones','Fecha','year','day', 'Problemas','Hora', 'FechaAux', 'Mantilla', 'Terreno', 'Tipo','Categoría', 'Meteorología', 'LLuvia', 'Viento', 'Temperatura', 'Hipodromo', 'Contrincantes']),
    ("scale", sklearn.preprocessing.StandardScaler(), []),
    # ("num", sklearn.preprocessing.MinMaxScaler(), ['Distancia', 'Edad', 'Peso']),
    ("one-hot", sklearn.preprocessing.OneHotEncoder(handle_unknown="ignore"), ['NombreCaballo','Propietario', 'Preparador', 'Jinete']),
], remainder='passthrough');

# column_transformer = sklearn.compose.ColumnTransformer(transformers=[
#     ("drop", "drop", ['UltimasActuaciones','Fecha','year','day', 'Problemas']),
#     ("scale", sklearn.preprocessing.StandardScaler(), []),
#     ("one-hot", sklearn.preprocessing.OneHotEncoder(handle_unknown="ignore"), ['NombreCaballo','Peso','Edad', 'Mantilla','Propietario', 'Preparador', 'Jinete', 'estacion', 'FechaAux', 'Terreno','Distancia', 'Tipo','Categoría', 'Meteorología', 'LLuvia', 'Viento', 'TemperaturaMax', 'TemperaturaMin', 'Hipodromo']),
# ], remainder='passthrough');

In [ ]:
train = train.reset_index()

X_val = column_transformer_test.fit_transform(X_val)
# X_test_transform = column_transformer.transform(X_val)

In [ ]:
column_transformer_train.get_feature_names()

In [ ]:
column_transformer_test.get_feature_names()

In [ ]:
from sklearn.metrics import mean_absolute_error

print('Lineal (train): ' + str(mean_absolute_error(y_train, linear.predict(X_train))))
predicted_y1 = linear.predict(X_val)
print('Lineal (test): ' + str(mean_absolute_error(predicted_y1, linear.predict(X_val))))


print('---')

print('Ridge (train): ' + str(mean_absolute_error(y_train, ridge.predict(X_train))))
predicted_y2 = ridge.predict(X_val)
print('Ridge (test): ' + str(mean_absolute_error(predicted_y2, ridge.predict(X_val))))

print('---')

print('LASSO (train): ' + str(mean_absolute_error(y_train, lasso.predict(X_train))))
predicted_y3 = lasso.predict(X_val)

print('LASSO (test): ' + str(mean_absolute_error(predicted_y3, lasso.predict(X_val))))

print('---')

print('ElasticNet (train): ' + str(mean_absolute_error(y_train, elasticnet.predict(X_train))))
predicted_y4 = elasticnet.predict(X_val)
print('ElasticNet (test): ' + str(mean_absolute_error(predicted_y4, elasticnet.predict(X_val))))

In [ ]:
y_val = pd.concat([train['index'],train['NombreCaballo']],axis=1)

y_val

In [ ]:
predicted_y = xgb_model.predict(X_val)
y_val['Puesto'] = predicted_y
y_val.to_csv('output1.csv' , index=False)

In [ ]:

predicted_y = model_cat_tun.predict(X_val)
y_val['Puesto'] = predicted_y
y_val.to_csv('output1.csv' , index=False)

In [ ]:

y_val['Puesto'] = predicted_y1
y_val.to_csv('output1.csv' , index=False)
y_val['Puesto'] = predicted_y2
y_val.to_csv('output2.csv', index=False)
y_val['Puesto'] = predicted_y3
y_val.to_csv('output3.csv', index=False)
y_val['Puesto'] = predicted_y4
y_val.to_csv('output4.csv' , index=False)

In [ ]:
pd.set_option('display.max_columns', None)
y_val.head(40)

- **Considero importante realizar una explicacion en cada subida de .csv sobre en que se diferencia respecto al anterior**